In [50]:
from Users.project.src.data_container.data_container import AzureStorageAccess

import pandas as pd
import numpy as np
from joblib import Parallel, delayed
data_access = AzureStorageAccess()

In [51]:
# 이 딥러닝에 필요한 클래스들 정의
def get_weather_data(current_time, data_frame: pd.DataFrame):
    last_data = data_frame[data_frame["Time"] <= current_time].tail(1)
    return last_data

def get_car_data_feature(data_frame: pd.DataFrame) -> list[float]:
    # 데이터 추출
    rpm_datas = data_frame["RPM"]
    speed_datas = data_frame["Speed"]
    gear_datas = data_frame["nGear"]
    throttle_datas = data_frame["Throttle"]
    brake_datas = data_frame["Brake"]
    drs_datas = data_frame["DRS"]
    # 피처 리스트 초기화
    features = []
    # 1. RPM 피처
    features.append(rpm_datas.mean())  # 평균 RPM
    features.append(rpm_datas.max())   # 최대 RPM
    features.append(rpm_datas.diff().abs().mean())  
    features.append(rpm_datas.std())   # RPM 표준편차 (변동성)
    # 2. Speed 피처
    features.append(speed_datas.mean())  # 평균 속도
    features.append(speed_datas.max())   # 최대 속도
    features.append(speed_datas.diff().abs().mean())  # 평균 속도 변화율 (가속도)
    features.append(speed_datas.std())   # 속도 표준편차
    # 3. nGear 피처
    features.append(gear_datas.mean()) 
    features.append(gear_datas.max())  
    features.append(gear_datas.diff().abs().mean())  # 평균 기어 변화율
    features.append(gear_datas.std()) 
    features.append(gear_datas.value_counts().max())  # 가장 많이 사용된 기어의 빈도
    features.append((gear_datas.shift() != gear_datas)[1:].sum())  # 기어 전환 횟수
    # 4. Throttle 피처
    features.append(throttle_datas.mean())  # 평균 스로틀 사용
    features.append(throttle_datas.max())   
    features.append(throttle_datas.diff().abs().mean()) 
    features.append(throttle_datas.std()) 
    features.append((throttle_datas > 0.95).sum() / len(throttle_datas)) # 거의 풀 스로틀의 비율
    # 5. Brake 피처
    features.append(brake_datas.mean())  # 브레이크 사용 비율
    # 6. DRS 피처
    features.append(drs_datas.mean())  # DRS 평균
    features.append(drs_datas.max())  # DRS 최대
    features.append(drs_datas.diff().abs().mean())  # 평균 drs 변화율
    features.append(drs_datas.std())  
    features.append(drs_datas.value_counts().max())  # 가장 많이 사용된 drs 빈도
    features.append((drs_datas.shift() != drs_datas)[1:].sum())  # drs 전환 횟수
#       코스팅(Coasting) 시간 비율: 스로틀과 브레이크를 모두 사용하지 않는 타력 주행 구간의 비율입니다. 드라이버의 효율성을 나타내는 지표가 될 수 있습니다.
#       # Throttle과 Brake가 모두 5% 미만인 시간의 비율
#       ((throttle_datas < 0.05) & (brake_datas < 0.05)).sum() / len(data_frame)
    return features

def get_laps_data_feature(row: pd.DataFrame) -> list[float]:
    stint = row.Stint
    tyre_life = row.TyreLife
    lap_number = row.LapNumber
    sector1_time = pd.to_timedelta(row.Sector1Time).total_seconds()
    sector2_time = pd.to_timedelta(row.Sector2Time).total_seconds()
    sector3_time = pd.to_timedelta(row.Sector3Time).total_seconds()
    compound = row.Compound
    fresh_tyre = row.FreshTyre
    track_status = int(row.TrackStatus)
    features = []
    # 타이어 상태 원핫인코딩
    tyre_mapping = {"SOFT": [1, 0, 0], "MEDIUM": [0, 1, 0], "HARD": [0, 0, 1]}
    compound_encoded = tyre_mapping.get(compound, [1, 0, 0])  # 알 수 없는 타이어는 [0,0,0]
    track_flags = []
    flag = 16  # 2^4부터 시작 (5비트)
    while flag != 0:
        track_flags.append(1.0 if track_status & flag else 0.0)
        flag //= 2
    features.append(fresh_tyre)
    features.append(stint)
    features.append(tyre_life)
    features.append(lap_number)
    features.append(sector1_time)
    features.append(sector2_time)
    features.append(sector3_time)
    
    features += compound_encoded
    #features += track_flags
    return features

def get_weather_data_feature(data_frame: pd.DataFrame) -> list[float]:
    air_temp = data_frame["AirTemp"].item()
    humidity = data_frame["Humidity"].item()
    pressure = data_frame["Pressure"].item()
    rainfall = data_frame["Rainfall"].item()
    track_temp = data_frame["TrackTemp"].item()
    wind_direction = data_frame["WindDirection"].item()
    wind_speed = data_frame["WindSpeed"].item()
    features = [air_temp, humidity, pressure, rainfall, track_temp, wind_direction, wind_speed]
    return features

def arrange_feature_and_label_has_nan(car_data: pd.DataFrame, laps_data: pd.DataFrame, weather_data: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    car_data["Time"] = pd.to_timedelta(car_data["Time"])
    laps_data["LapTime"] = pd.to_timedelta(laps_data["LapTime"])
    weather_data["Time"] = pd.to_timedelta(weather_data["Time"])
    progress_time = pd.Timedelta(0)
    one_team_features = []
    one_team_label = []
    for row in laps_data.itertuples():
        lap_number = row.LapNumber
        is_accurate = row.IsAccurate
        if is_accurate == False:
            continue
        track_status = row.TrackStatus
        if track_status != 1:
            continue
        lap_time = row.LapTime
        progress_time += lap_time
        same_lap_number_data = car_data[car_data["LapNumber"] == lap_number]
        if same_lap_number_data.empty or len(same_lap_number_data) < 2:
            continue
        weather_frame = get_weather_data(progress_time, weather_data)
        if weather_frame.empty:
            continue
        one_lap_features = []
        car_data_feature = get_car_data_feature(same_lap_number_data)
        laps_data_feature = get_laps_data_feature(row)
        weather_data_feature = get_weather_data_feature(weather_frame)
        one_lap_features += car_data_feature
        one_lap_features += laps_data_feature
        one_lap_features += weather_data_feature
        one_lap_features = [float(x) if isinstance(x, (np.float32, np.float64)) else x for x in one_lap_features]
        one_lap_features = [int(x) if isinstance(x, (np.int32, np.int64)) else x for x in one_lap_features]
        one_team_features.append(one_lap_features)
        one_team_label.append([lap_time.total_seconds()])  
    
    feature_headers = [
        # 1. RPM
        "rpm_mean", "rpm_max", "rpm_diff_mean", "rpm_std",
        # 2. Speed
        "speed_mean", "speed_max", "speed_diff_mean", "speed_std",
        # 3. nGear
        "gear_mean", "gear_max", "gear_diff_mean", "gear_std",
        "gear_most_freq", "gear_change_count",
        # 4. Throttle
        "throttle_mean", "throttle_max", "throttle_diff_mean", "throttle_std",
        "throttle_full_ratio",
        # 5. Brake
        "brake_mean",
        # 6. DRS
        "drs_mean", "drs_max", "drs_diff_mean", "drs_std",
        "drs_most_freq", "drs_change_count",
        # Laps Feature
        "fresh_tyre",
        "stint",
        "tyre_life",
        "lap_number",
        "sector1_time", "sector2_time", "sector3_time",
        "tyre_SOFT", "tyre_MEDIUM", "tyre_HARD",
        #"flag_16", "flag_8", "flag_4", "flag_2", "flag_1",  # 5비트
        # Weather Feature
        "air_temp", "humidity", "pressure", "rainfall", "track_temp",
        "wind_direction", "wind_speed"
    ]
    one_team_features = pd.DataFrame(one_team_features, columns=feature_headers)
    one_team_features["stint"] = one_team_features["stint"].ffill()
    one_team_features["tyre_life"] = one_team_features["tyre_life"].ffill()
    one_team_label = pd.DataFrame(one_team_label)
    return one_team_features, one_team_label


In [42]:
# for file in data_access.get_all_file():
#     file_name = file.name
#     if "laps.csv" not in file_name:
#         continue

#     data_frame = data_access.read_csv_from_blob(file_name)

#     # speed_st가 NaN인 행 필터링
#     nan_speed_rows = data_frame[data_frame["TyreLife"].isna()]

#     # 해당 행들의 laptime 출력
#     if not nan_speed_rows.empty:
#         print(f"{file_name} - NaN speed_st rows:")
#         print(nan_speed_rows["LapTime"])

In [ ]:
# import pandas as pd
# import os
# from joblib import Parallel, delayed
# # 사용자 정의 모듈들은 이미 임포트되어 있다고 가정
# # from Users.project.src... import data_access, arrange_feature_and_label_has_nan

# def remove_nan_pairs(features: pd.DataFrame, labels: pd.DataFrame):
#     # isna()전에 데이터 타입이 통일되지 않으면 에러가 발생할 수 있으므로 numeric으로 변환
#     features = features.apply(pd.to_numeric, errors='coerce')
#     valid_mask = ~features.isna().any(axis=1)
#     return features[valid_mask].reset_index(drop=True), labels[valid_mask].reset_index(drop=True)

# def process_game_team(game_team_name: str):
#     """
#     하나의 game_team_name에 대한 전체 처리 과정을 담은 '작업 함수'.
#     이 함수가 여러 CPU 코어에서 동시에 실행됩니다.
#     """
#     try:
#         # 1. 데이터 읽기
#         car_data = data_access.read_csv_by_data_frame(f"{game_team_name}car_data_all.csv")
#         laps_data = data_access.read_csv_by_data_frame(f"{game_team_name}laps.csv")
#         tokens = game_team_name.split("/")
#         game_name = f"{tokens[0]}/{tokens[1]}/"
#         weather_data = data_access.read_csv_by_data_frame(f"{game_name}weather_data.csv")

#         # 2. 피쳐링 및 NaN 처리
#         feature, label = arrange_feature_and_label_has_nan(car_data, laps_data, weather_data)
#         feature_df, label_df = remove_nan_pairs(feature, label)

#         # 3. 결과 반환
#         if feature_df.empty:
#             return None # 처리할 데이터가 없으면 None 반환
        
#         return feature_df, label_df
#     except Exception as e:
#         print(f"Error processing {game_team_name}: {e}")
#         return None # 에러 발생 시 None 반환

# # --- 메인 실행 로직 ---
# # multiprocessing을 안전하게 사용하기 위해 if __name__ == "__main__": 구문 안에 넣는 것이 좋습니다.
# if __name__ == "__main__":
#     file_folder = os.path.join("Users", "project", "correct_file", "lap_time_predict")
#     file_folder = os.path.abspath(file_folder)
#     save_folder = os.path.join("Users", "project", "data", "lap_time_predict")
#     # 1. 병렬 처리할 '작업 목록'을 미리 준비합니다.
#     all_game_team_names = []
#     for file_name in os.listdir(file_folder):
#         full_path = os.path.join(file_folder, file_name)
#         with open(full_path, "r", encoding="utf-8") as file:
#             for game_team_name_line in file:
#                 all_game_team_names.append(game_team_name_line.strip())

#     print(f"Total jobs to process: {len(all_game_team_names)}")

#     # 2. joblib.Parallel을 사용하여 병렬로 작업을 실행합니다.
#     # n_jobs=-1 은 사용 가능한 모든 CPU 코어를 사용하라는 의미입니다.
#     # delayed(함수)(인자) 형태로 작업을 등록합니다.
#     print("Starting parallel processing...")
#     results = Parallel(n_jobs=-1)(delayed(process_game_team)(name) for name in all_game_team_names)
#     print("Parallel processing finished.")

#     # 3. 결과들을 취합합니다.
#     # 결과 리스트(results)에는 (feature_df, label_df) 튜플 또는 None이 들어있습니다.
#     valid_results = [res for res in results if res is not None]

#     if valid_results:
#         # feature와 label을 각각의 리스트로 분리
#         features_list = [res[0] for res in valid_results]
#         labels_list = [res[1] for res in valid_results]

#         # 최종적으로 하나의 데이터프레임으로 합칩니다.
#         X = pd.concat(features_list, ignore_index=True)
#         y = pd.concat(labels_list, ignore_index=True)

#         print("\nFinal DataFrame shape:")
#         print(f"Features (X): {X.shape}")
#         print(f"Labels (y): {y.shape}")
#         print("X에 NaN 있음:", X.isna().values.any())
#         print("y에 NaN 있음:", y.isna().values.any())
#         file_count = len([
#             f for f in os.listdir(save_folder)
#             if os.path.isfile(os.path.join(save_folder, f))
#         ])
#         feature_path = os.path.join(save_folder, f"features{file_count}.csv")
#         label_path = os.path.join(save_folder, f"labels{file_count}.csv")
#         X.to_csv(feature_path, index=False)
#         y.to_csv(label_path, index=False)
#     else:
#         print("No valid data was processed.")

Total jobs to process: 14674
Starting parallel processing...
Parallel processing finished.

Final DataFrame shape:
Features (X): (269814, 43)
Labels (y): (269814, 1)
X에 NaN 있음: False
y에 NaN 있음: False


In [54]:
for data in data_access.get_all_file():
    if "laps.csv" not in data.name:
        continue

    df = data_access.read_csv_from_blob(data.name)

    # df = data_access.read_csv_from_blob(data.name) # <- 이 부분 다음
    df["LapTime"] = pd.to_timedelta(df["LapTime"])
    
    # 2. LapTime이 NaN이 된 행들을 데이터프레임에서 완전히 제거합니다.
    df.dropna(subset=['LapTime'], inplace=True)
    
    # 3. 이제 LapTime 컬럼은 깨끗한 숫자형 데이터만 가지고 있으므로,
    #    아래 그룹화 및 집계 연산이 정상적으로 작동합니다.
    status_analysis = df.groupby('TrackStatus')['LapTime'].agg(['mean', 'std', 'count', 'min', 'max'])
    
    # 보기 좋게 정렬
    status_analysis = status_analysis.sort_values(by='mean')
    
    print(status_analysis)

                              mean                       std  count  \
TrackStatus                                                           
12.0        0 days 00:01:26.285000                       NaT      1   
1.0         0 days 00:01:34.170400 0 days 00:00:09.579511646     10   

                               min                    max  
TrackStatus                                                
12.0        0 days 00:01:26.285000 0 days 00:01:26.285000  
1.0         0 days 00:01:25.896000 0 days 00:01:55.334000  
                                 mean                       std  count  \
TrackStatus                                                              
1.0         0 days 00:01:36.320263157 0 days 00:00:13.264885520     19   
21.0           0 days 00:01:44.453000                       NaT      1   

                               min                    max  
TrackStatus                                                
1.0         0 days 00:01:24.577000 0 days 00:02:02.941000 

KeyboardInterrupt: 